In [1]:
import pandas as pd

In [2]:
with open("../../data/curvibacter_annotation_files/translation_table.csv", "r") as tfile:
    with open("../../data/curvibacter_annotation_files/translation_table_corrected.csv","w") as corrtfile:
        lines = tfile.readlines()
        for line in lines:
            line = line.rstrip()
            currline = line.split("\t")
            if len(currline) == 3:
                corrtfile.write(line+"\n")

In [3]:
translation_table = pd.read_table("../../data/curvibacter_annotation_files/curvibacter_genome_table_go.csv")

In [4]:
translation_table.head()

,locus_tag,old_locus_tag,protein_id,GOfun,GOpro,GOfun_evidence,GOpro_evidence,product
0,AEP_RS00005,AEP_00001,WP_087493495.1,none,none,none,none,response regulator transcription factor
1,AEP_RS00010,AEP_00002,WP_087493496.1,GO:0009678,GO:1902600,IEA,IEA,sodium-translocating pyrophosphatase
2,AEP_RS00015,AEP_00003,WP_087493497.1,GO:0004427,GO:0006796,IEA,IEA,inorganic diphosphatase
3,AEP_RS00020,AEP_00004,WP_087493498.1,none,none,none,none,alpha/beta fold hydrolase
4,AEP_RS00025,AEP_00005,WP_087493499.1,none,GO:0006935;GO:0007165,none,IEA;IEA,chemotaxis protein CheW


In [5]:
gene_to_go_dict = {}
gene_to_goEvidence_dict = {}

for locus in translation_table.old_locus_tag:    
    gene_to_go_dict[locus] = []
    gene_to_goEvidence_dict[locus] = []
    
    gopro_list = translation_table[translation_table.old_locus_tag == locus].GOpro.apply(lambda x: x.split(";")).to_list()[0]
    gofun_list = translation_table[translation_table.old_locus_tag == locus].GOfun.apply(lambda x: x.split(";")).to_list()[0]
    gopro_evidence_list = translation_table[translation_table.old_locus_tag == locus].GOpro_evidence.apply(lambda x: x.split(";")).to_list()[0]
    gofun_evidence_list = translation_table[translation_table.old_locus_tag == locus].GOfun_evidence.apply(lambda x: x.split(";")).to_list()[0]

    if gopro_list[0] != "none":
        for gopro in gopro_list:
            gene_to_go_dict[locus].append(gopro)
        for gopro_evidence in gopro_evidence_list:
            gene_to_goEvidence_dict[locus].append(gopro_evidence)

    if gofun_list[0] != "none":
        for gofun in gofun_list:
            gene_to_go_dict[locus].append(gofun)
        
        for gofun_evidence in gofun_evidence_list:
            gene_to_goEvidence_dict[locus].append(gofun_evidence)

In [6]:
with open("../../data/curvibacter_annotation_files/curvibacter_go_annotation_forge.csv","w") as goannot:
    goannot.write("GID\tGO\tEVIDENCE\n")
    for key in gene_to_go_dict.keys():
        if len(gene_to_go_dict[key]) > 0:
            for go, evidence in zip(gene_to_go_dict[key],gene_to_goEvidence_dict[key]):
                goannot.write(key + "\t" + go + "\t" + evidence + "\n")        

In [7]:
# TODO add genes on plasmid ...
with open("../../data/curvibacter_annotation_files/curvibacter_chromosome_annotation_forge.csv","w") as chromosome_annot:
    chromosome_annot.write("GID\tCHROMOSOME\n")
    for key in translation_table.old_locus_tag:
        chromosome_annot.write(key + "\t" + "1\n")

In [8]:
# TODO add genes on plasmid ...
with open("../../data/curvibacter_annotation_files/curvibacter_info_annotation_forge.csv","w") as gene_info:
    gene_info.write("GID\tSYMBOL\tGENENAME\n")
    for gid, gsymbol, ginfo in zip(translation_table.old_locus_tag, translation_table.protein_id, translation_table["product"]):
        gene_info.write(gid + "\t" + gsymbol + "\t" + ginfo + "\n")